In [1]:
import matplotlib as mpl
from lxml import etree

In [2]:
start_year = 1985
end_year = 2023

# Interpolate the inferno colour ramp between the two years
cmap = mpl.colormaps["inferno"]
norm = mpl.colors.Normalize(vmin=start_year, vmax=end_year)

# Get the values from the colour ramp as hex
hex_values = {}
for i in range(start_year, end_year + 1):
    rgb = cmap(norm(i))
    hex_values[i] = mpl.colors.rgb2hex(rgb)

In [3]:
# Open the base
base = """
<sld:StyledLayerDescriptor xmlns="http://www.opengis.net/sld" xmlns:sld="http://www.opengis.net/sld" xmlns:gml="http://www.opengis.net/gml" xmlns:ogc="http://www.opengis.net/ogc" version="1.0.0">
    <sld:NamedLayer>
        <sld:Name>coastlines_shorelines_annual</sld:Name>
        <sld:UserStyle>
            <sld:Name>coastlines_shorelines_annual</sld:Name>
            <sld:FeatureTypeStyle>
                <sld:Name>name</sld:Name>
                <sld:VendorOption name="sortBy">year</sld:VendorOption>
            </sld:FeatureTypeStyle>
        </sld:UserStyle>
    </sld:NamedLayer>
</sld:StyledLayerDescriptor>
"""

tree = etree.fromstring(base)

In [4]:
one_rule = """
                <sld:Rule xmlns="http://www.opengis.net/sld" xmlns:sld="http://www.opengis.net/sld" xmlns:gml="http://www.opengis.net/gml" xmlns:ogc="http://www.opengis.net/ogc">
                    <sld:Name>{year}</sld:Name>
                    <sld:Title>{year} ({quality} quality shorelines)</sld:Title>
                    <ogc:Filter>
                        <ogc:And>
                            <ogc:Property{is_isnot}EqualTo>
                                <ogc:PropertyName>certainty</ogc:PropertyName>
                                <ogc:Literal>good</ogc:Literal>
                            </ogc:Property{is_isnot}EqualTo>
                            <ogc:PropertyIsEqualTo>
                                <ogc:PropertyName>year</ogc:PropertyName>
                                <ogc:Literal>{year}</ogc:Literal>
                            </ogc:PropertyIsEqualTo>
                        </ogc:And>
                    </ogc:Filter>
                    <sld:MaxScaleDenominator>50000.0</sld:MaxScaleDenominator>
                    <sld:LineSymbolizer>
                        <sld:Stroke>
                            <sld:CssParameter name="stroke">{color}</sld:CssParameter>
                            <sld:CssParameter name="stroke-linecap">square</sld:CssParameter>
                            <sld:CssParameter name="stroke-linejoin">bevel</sld:CssParameter>
                            {stroke}
                        </sld:Stroke>
                    </sld:LineSymbolizer>
                </sld:Rule>
"""

good_stroke = """<sld:CssParameter name="stroke-width">2</sld:CssParameter>"""
bad_stroke = """<sld:CssParameter name="stroke-dasharray">5.0 3.5</sld:CssParameter>"""

# Add a bunch of rules in after sld:FeatureTypeStyle
feature_type_style = tree.find(".//sld:FeatureTypeStyle", namespaces=tree.nsmap)

for certainty in ["good", "bad"]:
    for year, color in hex_values.items():
        rule = one_rule.format(
            year=year,
            quality=certainty,
            is_isnot="Is" if certainty == "good" else "IsNot",
            color=color,
            stroke=good_stroke if certainty == "good" else bad_stroke,
        )

        rule_xml = etree.fromstring(rule)
        feature_type_style.append(rule_xml)

In [5]:
# Labels
final_rule = """
        <sld:Rule xmlns="http://www.opengis.net/sld" xmlns:sld="http://www.opengis.net/sld" xmlns:gml="http://www.opengis.net/gml" xmlns:ogc="http://www.opengis.net/ogc">
            <sld:MaxScaleDenominator>10000.0</sld:MaxScaleDenominator>
            <sld:TextSymbolizer>
                <sld:Label>
                    <ogc:PropertyName>year</ogc:PropertyName>
                </sld:Label>
                <sld:Font>
                    <sld:CssParameter name="font-family">SansSerif.plain</sld:CssParameter>
                    <sld:CssParameter name="font-size">13</sld:CssParameter>
                    <sld:CssParameter name="font-style">normal</sld:CssParameter>
                    <sld:CssParameter name="font-weight">normal</sld:CssParameter>
                </sld:Font>
                <sld:LabelPlacement>
                    <sld:LinePlacement />
                </sld:LabelPlacement>
                <sld:Halo>
                    <sld:Radius>2</sld:Radius>
                    <sld:Fill>
                        <sld:CssParameter name="fill">#000000</sld:CssParameter>
                        <sld:CssParameter name="fill-opacity">0.477</sld:CssParameter>
                    </sld:Fill>
                </sld:Halo>
                <sld:Fill>
                    <sld:CssParameter name="fill">#ffffff</sld:CssParameter>
                </sld:Fill>
            </sld:TextSymbolizer>
        </sld:Rule>
"""

# Add the final_rule to the list of <sld:Rule> inside </sld:FeatureTypeStyle>
final_rule_xml = etree.fromstring(final_rule)
feature_type_style.append(final_rule_xml)

In [6]:
# Write it out
with open("coastlines_shorelines_annual.sld", "wb") as f:
    f.write(etree.tostring(tree, pretty_print=True))